In [1]:
import findspark 
findspark.init()

In [2]:
from pyspark.conf import SparkConf
config = SparkConf()
config.setMaster("spark://192.168.11.71:7077").setAppName("DFJoin")
config.set("spark.executor.memory", "4g")
config.set("spark.executor.cores", 2)
config.set("spark.cores.max", 2)
config.set("spark.driver.memory", "4g")

from pyspark.sql import SparkSession
spark = SparkSession.builder.config(conf=config).getOrCreate()

22/05/09 22:06:10 WARN Utils: Your hostname, ubuntu-virtual-machine resolves to a loopback address: 127.0.1.1; using 192.168.174.129 instead (on interface ens33)
22/05/09 22:06:11 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
22/05/09 22:06:11 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
Using Spark's default log4j profile: org/apache/spark/log4j-defaults.properties
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
22/05/09 22:06:15 WARN Utils: Service 'SparkUI' could not bind on port 4040. Attempting port 4041.
22/05/09 22:06:15 WARN Utils: Service 'SparkUI' could not bind on port 4041. Attempting port 4042.
22/05/09 22:06:15 WARN Utils: Service 'SparkUI' could not bind on port 4042. Attempting port 4043.
22/05/09 22:06:15 WARN Utils: Service 'SparkUI' could not bind on port 4043. Attempting port 4044.
2

Py4JJavaError: An error occurred while calling None.org.apache.spark.api.java.JavaSparkContext.
: java.lang.IllegalArgumentException: requirement failed: Can only call getServletHandlers on a running MetricsSystem
	at scala.Predef$.require(Predef.scala:281)
	at org.apache.spark.metrics.MetricsSystem.getServletHandlers(MetricsSystem.scala:92)
	at org.apache.spark.SparkContext.<init>(SparkContext.scala:597)
	at org.apache.spark.api.java.JavaSparkContext.<init>(JavaSparkContext.scala:58)
	at sun.reflect.NativeConstructorAccessorImpl.newInstance0(Native Method)
	at sun.reflect.NativeConstructorAccessorImpl.newInstance(NativeConstructorAccessorImpl.java:62)
	at sun.reflect.DelegatingConstructorAccessorImpl.newInstance(DelegatingConstructorAccessorImpl.java:45)
	at java.lang.reflect.Constructor.newInstance(Constructor.java:423)
	at py4j.reflection.MethodInvoker.invoke(MethodInvoker.java:247)
	at py4j.reflection.ReflectionEngine.invoke(ReflectionEngine.java:357)
	at py4j.Gateway.invoke(Gatewa

In [3]:
from pyspark.conf import SparkConf
config = SparkConf()
config.setMaster("local").setAppName("DFJoin")


from pyspark.sql import SparkSession
spark = SparkSession.builder.config(conf=config).getOrCreate()

22/05/09 22:08:08 WARN SparkContext: Another SparkContext is being constructed (or threw an exception in its constructor). This may indicate an error, since only one SparkContext should be running in this JVM (see SPARK-2243). The other SparkContext was created at:
org.apache.spark.api.java.JavaSparkContext.<init>(JavaSparkContext.scala:58)
sun.reflect.NativeConstructorAccessorImpl.newInstance0(Native Method)
sun.reflect.NativeConstructorAccessorImpl.newInstance(NativeConstructorAccessorImpl.java:62)
sun.reflect.DelegatingConstructorAccessorImpl.newInstance(DelegatingConstructorAccessorImpl.java:45)
java.lang.reflect.Constructor.newInstance(Constructor.java:423)
py4j.reflection.MethodInvoker.invoke(MethodInvoker.java:247)
py4j.reflection.ReflectionEngine.invoke(ReflectionEngine.java:357)
py4j.Gateway.invoke(Gateway.java:238)
py4j.commands.ConstructorCommand.invokeConstructor(ConstructorCommand.java:80)
py4j.commands.ConstructorCommand.execute(ConstructorCommand.java:69)
py4j.GatewayCon

In [4]:
products = [ 
          # (product_id, product_name, brand_id)  
         (1, 'iPhone', 100),
         (2, 'Galaxy', 200),
         (3, 'Redme', 300), # orphan record, no matching brand
         (4, 'Pixel', 400),
]

brands = [
    #(brand_id, brand_name)
    (100, "Apple"),
    (200, "Samsung"),
    (400, "Google"),
    (500, "Sony"), # no matching products
]
 
productDf = spark.createDataFrame(data=products, schema=["product_id", "product_name", "brand_id"])
brandDf = spark.createDataFrame(data=brands, schema=["brand_id", "brand_name"])
productDf.show()
brandDf.show()

+----------+------------+--------+
|product_id|product_name|brand_id|
+----------+------------+--------+
|         1|      iPhone|     100|
|         2|      Galaxy|     200|
|         3|       Redme|     300|
|         4|       Pixel|     400|
+----------+------------+--------+

+--------+----------+
|brand_id|brand_name|
+--------+----------+
|     100|     Apple|
|     200|   Samsung|
|     400|    Google|
|     500|      Sony|
+--------+----------+



In [6]:
# Inner Join
# productDf is left
# brandDf is right
# select/pick only matching record, discord if no matches found
df = productDf.join(brandDf, productDf["brand_id"] ==  brandDf["brand_id"], "inner")

In [7]:
# Outer Join, Full Outer Outer, [Left outer + Right outer]
# pick all records from left dataframe, and also right dataframe
# if no matches found, it fills null data for not matched records
productDf.join(brandDf, productDf["brand_id"] ==  brandDf["brand_id"], "outer").show()

+----------+------------+--------+--------+----------+
|product_id|product_name|brand_id|brand_id|brand_name|
+----------+------------+--------+--------+----------+
|      null|        null|    null|     500|      Sony|
|         1|      iPhone|     100|     100|     Apple|
|         2|      Galaxy|     200|     200|   Samsung|
|         4|       Pixel|     400|     400|    Google|
|         3|       Redme|     300|    null|      null|
+----------+------------+--------+--------+----------+



In [12]:
df = productDf.join(brandDf, productDf["brand_id"] ==  brandDf["brand_id"], "inner")\
    .drop("brand_id")


In [13]:
df.printSchema()

root
 |-- product_id: long (nullable = true)
 |-- product_name: string (nullable = true)
 |-- brand_name: string (nullable = true)



In [14]:
df.show()


+----------+------------+----------+
|product_id|product_name|brand_name|
+----------+------------+----------+
|         1|      iPhone|     Apple|
|         2|      Galaxy|   Samsung|
|         4|       Pixel|    Google|
+----------+------------+----------+



In [15]:
# Left, Left Outer join 
# picks all records from left, if no matches found, it fills null for right data
productDf.join(brandDf, productDf["brand_id"] ==  brandDf["brand_id"], "leftouter").show()

+----------+------------+--------+--------+----------+
|product_id|product_name|brand_id|brand_id|brand_name|
+----------+------------+--------+--------+----------+
|         1|      iPhone|     100|     100|     Apple|
|         2|      Galaxy|     200|     200|   Samsung|
|         4|       Pixel|     400|     400|    Google|
|         3|       Redme|     300|    null|      null|
+----------+------------+--------+--------+----------+



In [16]:
# Right, Right outer Join
# picks all the records from right, if no matches found, fills left data with null
productDf.join(brandDf, productDf["brand_id"] ==  brandDf["brand_id"], "rightouter").show()

+----------+------------+--------+--------+----------+
|product_id|product_name|brand_id|brand_id|brand_name|
+----------+------------+--------+--------+----------+
|      null|        null|    null|     500|      Sony|
|         1|      iPhone|     100|     100|     Apple|
|         2|      Galaxy|     200|     200|   Samsung|
|         4|       Pixel|     400|     400|    Google|
+----------+------------+--------+--------+----------+



In [17]:
# left semi join
# join in general convention, it pull the records from both right and left, join them based on condition
# left semi join, join left and right based on condition, however it pull the records only from left side

# it is similar to innerjoin, but pick/project records only from left
# we can't see brand_id, brand_name from brands df
productDf.join(brandDf, productDf["brand_id"] ==  brandDf["brand_id"], "leftsemi").show()

+----------+------------+--------+
|product_id|product_name|brand_id|
+----------+------------+--------+
|         1|      iPhone|     100|
|         2|      Galaxy|     200|
|         4|       Pixel|     400|
+----------+------------+--------+



In [18]:
# left anti join: exact opposite to semi join
# picks the records that doesn't have match on the right side
productDf.join(brandDf, productDf["brand_id"] ==  brandDf["brand_id"], "leftanti").show()

+----------+------------+--------+
|product_id|product_name|brand_id|
+----------+------------+--------+
|         3|       Redme|     300|
+----------+------------+--------+



In [19]:
store = [
    #(store_id, store_name)
    (1000, "Poorvika"),
    (2000, "Sangeetha"),
    (4000, "Amazon"),
    (5000, "FlipKart"), 
]
 
storeDf = spark.createDataFrame(data=store, schema=["store_id", "store_name"])
storeDf.show()


+--------+----------+
|store_id|store_name|
+--------+----------+
|    1000|  Poorvika|
|    2000| Sangeetha|
|    4000|    Amazon|
|    5000|  FlipKart|
+--------+----------+



In [20]:
# cartesian , take row from left side, pair with all from right side
productDf.crossJoin(storeDf).show()


+----------+------------+--------+--------+----------+
|product_id|product_name|brand_id|store_id|store_name|
+----------+------------+--------+--------+----------+
|         1|      iPhone|     100|    1000|  Poorvika|
|         1|      iPhone|     100|    2000| Sangeetha|
|         1|      iPhone|     100|    4000|    Amazon|
|         1|      iPhone|     100|    5000|  FlipKart|
|         2|      Galaxy|     200|    1000|  Poorvika|
|         2|      Galaxy|     200|    2000| Sangeetha|
|         2|      Galaxy|     200|    4000|    Amazon|
|         2|      Galaxy|     200|    5000|  FlipKart|
|         3|       Redme|     300|    1000|  Poorvika|
|         3|       Redme|     300|    2000| Sangeetha|
|         3|       Redme|     300|    4000|    Amazon|
|         3|       Redme|     300|    5000|  FlipKart|
|         4|       Pixel|     400|    1000|  Poorvika|
|         4|       Pixel|     400|    2000| Sangeetha|
|         4|       Pixel|     400|    4000|    Amazon|
|         